In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px # Pour installer le module Plotly, exécuter la commande ci-dessous
import plotly.graph_objects as go
#from tabulate import tabulate

import random
import sys
import seaborn as sns
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE, RFECV, SelectKBest, f_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
IN_COLAB = 'google.colab' in sys.modules

import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 100)

L'univers regorge d'information. Elle constitue un langage universel qui transcende les disciplines : des sonnets de Shakespeare aux articles scientifiques sur ArXiv, des coups de pinceau de *La Nuit étoilée* de Van Gogh aux algorithmes d'apprentissage automatique les plus avancés. La théorie de l'information permet de mesurer et comparer la quantité d'information contenue dans un signal. Dans cette section, nous explorerons ses concepts fondamentaux et ses applications en apprentissage automatique.

### La théorie de l'information
Introduite par **Claude Shannon**, la théorie de l'information utilise les probabilités pour quantifier le contenu moyen en information d'un ensemble de messages, lorsque leur codage respecte une distribution statistique connue. Cette théorie est essentielle pour la quantification du stockage et de la transmission de l'information. Elle trouve des applications majeures dans les télécommunications, la compression des données, la cryptographie, et bien d'autres domaines.

### Comprendre l'entropie
L'**information** peut être encodée dans tout système utilisant une séquence séquences de variables aléatoires. Mais qu'est-ce que l'information ? Considérons l'exemple d'un lancer de dé.

Un dé standard à six faces produit six résultats possibles : 1, 2, 3, 4, 5 ou 6. Avant le lancer, chaque face a une probabilité égale de 1/6. Vous n'avez donc aucune idée précise du résultat. Cette incertitude, est mesurée par l'**entropie**.  Dans ce cas-ci, on dira que **l'entropie est maximale car l'information est minimale**.

### Réduction de l'incertitude et de l'entropie grâce à l'information
Supposons maintenant qu'une personne voit le résultat du lancer mais ne vous le révèle pas complètement. Par contre, elle vous donne un indice :

- le résultat du dé est un **nombre pair**.  Cela vous permet d'éliminez les résultats 1, 3 et 5 et d'en déduire que le résultat ne peut être que 2, 4 ou 6. Votre incertitude aura alors diminué, car seulement trois options sont désormais possibles.  Dans ce cas, l'ajout d'information a réduit votre incertitude ainsi que l'entropie.
- Si la personne ajoute que le résultat est supérieur à 4, vous savez avec certitude qu'il s'agit de 6. Dans ce cas, votre incertitude ainsi que l'entropie sont nulles alors l'information est maximale.

En résumé, l'**entropie** mesure l'**incertitude** quant à la prédiction d'une variable aléatoire, tandis que l'**information** reçue réduit cette incertitude. Plus l'information est précise, plus l'entropie diminue, jusqu'à disparaître complètement lorsque le résultat est parfaitement connu.

## Information individuelle


Puisque l'information reflète la possibilité abstraite d'un événement, comment traduire cette possibilité en nombre de bits ? **Claude Shannon** a introduit le terme « bit » comme unité d'information, un concept inventé à l'origin¨e par **John Tukey**.  Historiquement, les premières machines pouvaient uniquement envoyer ou recevoir deux types de signaux : 0 et 1. Ce système de codage binaire est resté par la suite la norme dans tous les ordinateurs. Toute information est donc représentée sous forme de séquences de 0 et 1, et une séquence binaire de longueur $n$ contient $n$ bits d'information.

### Lien entre information et probabilité

Il existe un lien mathématique entre l'information et la probabilité.  On défini cela comme étant le principe **d'information individuelle**.  Ce principe s'appui sur l'idée que plus un événement est rare, plus l'information qu'il contient est élevée. Cela est formalisé par la formule :
$$
I(X) = -\log_2(p),
$$
où $p$ est la probabilité de l'événement $X$ et l'unité de mesure est un **bit**.   Ainsi, si on suppose que dans une série de caractères binaires 0 et 1, chaque caractère se produit avec une probabilité de $\frac{1}{2}$, on dira qu'un caractète possède $I(X) = -\log_2(\frac{1}{2})=1$ bit.  Dans le cas d'une séquence binaire « 0010 », sa probabilité état de $\frac{1}{2^4}$, l'information qu'il contient est :
$$
I(\textrm{"0010"}) = -\log_2(p(\textrm{"0010"})) = -\log_2\left(\frac{1}{2^4}\right) = 4 \, \textrm{bits}.
$$




In [ ]:
# Définition des probabilités
events = ['A', 'B', 'C', 'D']
probabilities = np.array([1/10, 2/10, 4/10, 3/10])  # Les proportions pour A, B, C, D

n = 1000 # Taille de l'array
f_data = np.random.choice(events, size=n, p=probabilities)

# Calcul de l'information individuelle
information = -np.log2(probabilities)

# Affichage de l'information individuelle
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.bar(events, probabilities)
plt.title('Distribution de probabilité')

# Mettre une vraie distribution de probabilité 
plt.subplot(1, 2, 2)
plt.bar(events, information, color='skyblue')
plt.xlabel('Événements')
plt.ylabel('Information (bits)')
plt.title('Information Individuelle')


plt.show()


## Entropie

L'*entropie* (ou *Entropie de Shannon*) mesure la quantité moyenne d'information associée à une variable aléatoire $x$ suivant une distribution de probabilité $p(x)$. Elle se définit comme :

$$H(X) = - E_{x \sim P} [\log p(x)],$$

où $E$ désigne l'espérance mathématique. En fonction de la nature de $X$, la formule s'adapte :

- Si $X$ est une variable continue :
  $$H(X) = - \int_x p(x) \log p(x) \; dx.$$

- Si $X$ est une variable discrète :
  $$H(X) = - \sum_i p(x_i) \log p(x_i).$$

L'entropie représente donc l'information moyenne contenue dans une ou plusieurs variables aléatoires. Elle est maximale lorsque **tous les événements sont équiprobables**, c'est-à-dire lorsque l'incertitude est à son apogée.

### Exemple de calcul
Nous allons maintenant calculer l'entropie en utilisant la même distribution que celle évoquée précédemment.


In [ ]:
# Calcul de l'entropie
entropy = np.sum(probabilities * information)

print(f'Entropie calculée: {entropy:.4f} bits')

## Exercice

1. Retournez à la cellule précédente et remplacez la distribution de probabilité **[1/10, 2/10, 4/10, 3/10]** par une distribution uniforme **[1/4, 1/4, 1/4, 1/4]** (c'est-à-dire une distribution avec une incertitude maximale). Observez l'effet sur l'entropie. Est-ce que l'entropie augmente ou diminue ?

2. Ensuite, remplacez cette distribution par une distribution totalement certaine, telle que **[0, 1, 0, 0]**. Que se passe-t-il avec l'entropie ? Comparez les résultats pour comprendre comment l'entropie varie en fonction de la distribution de probabilité.


## Codage de Huffman

Le **codage de Huffman** est une méthode de compression sans perte qui permet de réduire la quantité de bits nécessaires pour représenter une information. Il est particulièrement efficace lorsqu'il existe une distribution inégale des symboles dans une séquence, c'est-à-dire lorsque certains symboles apparaissent plus fréquemment que d'autres.

### Principe
Le codage de Huffman repose sur un algorithme qui attribue des codes binaires de longueur variable aux symboles. Les symboles les plus fréquents reçoivent des codes courts, tandis que les symboles moins fréquents se voient attribuer des codes plus longs. Ce mécanisme optimise la représentation des données minimisant ainsi le nombre total de bits nécessaires.

### Applications
Le codage de Huffman est utilisé dans de nombreux domaines, notamment :
- **Compression de fichiers** : Formats comme JPEG et MP3.
- **Algorithmes de compression de données** : Par exemple, l'algorithme DEFLATE, utilisé par ZIP et gzip.

### Avantages
- **Compression efficace** : Idéal pour les données où les fréquences des symboles varient considérablement.
- **Sans perte** : Aucune information n'est perdue durant la compression, garantissant une récupération parfaite des données.

### Limitations
- **Analyse préalable requise** : La fréquence des symboles doit être connue avant l'encodage, ce qui peut être coûteux pour de grandes quantités de données.
- **Efficacité limitée pour des fréquences similaires** : Si les symboles ont des fréquences proches, les gains de compression seront réduits.

Essayons à présent d'encoder le texte "*Voici un exemple pour illustrer le codage de Huffman*" en utilisant le codage de Huffman.  Mais avant, veuillez remarquez que si vous stockez cette phrase contenant **52 caractères** avec des cararactères ascii à 8 bits chacun, il vous faudra utiliser **52 x 8 = 416** bits au total.

In [ ]:
# Librement inspiré de : https://gist.github.com/jmbhughes/920a29432039ad7f8b656ca2931683e9

from heapq import heappush, heappop, heapify
from collections import Counter

def huffman_coding(symbols):
    heap = [[weight, [symbol, ""]] for symbol, weight in symbols.items()]
    heapify(heap)
    while len(heap) > 1:
        lo = heappop(heap)
        hi = heappop(heap)
        for pair in lo[1:]:
            pair[1] = '0' + pair[1]
        for pair in hi[1:]:
            pair[1] = '1' + pair[1]
        heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])
    return sorted(heappop(heap)[1:], key=lambda p: (len(p[-1]), p))

In [ ]:
text = "Voici un exemple pour illustrer le codage de Huffman"
frequency = Counter(text)
huff_code = huffman_coding(frequency)

print("Symbol\tWeight\tHuffman Code")
for p in huff_code:
    print(f"{p[0]}\t{frequency[p[0]]}\t{p[1]}")

Observez que les symbols "SPACE", "e" et "u" sont les plus fréquents et se sont vu attribués 3 bits.  Au contraire, les symbols "H", "V", "g", "s" étant les moins fréquents, ils se sont vus attribués 6 bits.

In [ ]:
# Affichage des codes de Huffman
symbols, codes = zip(*[(p[0], p[1]) for p in huff_code])

# Calcul des fréquences
freq = []
txt_lg = len(text)
for p in huff_code:
    freq.append(frequency[p[0]]/txt_lg)

plt.figure(figsize=(20, 5))

plt.subplot(1, 3, 1)
plt.bar(symbols, freq)
plt.xlabel('Symboles')
plt.ylabel('Probabilités')
plt.title('Distribution de probabilité (p)')

plt.subplot(1, 3, 2)
inf = -np.log2(freq)
plt.bar(symbols, inf, color='skyblue')
plt.xlabel('Symboles')
plt.ylabel('Information (bits)')
plt.title('Information Individuelle (-log(p))')

plt.subplot(1, 3, 3)
plt.bar(symbols, [len(code) for code in codes], color='lightgreen')
plt.xlabel('Symboles')
plt.ylabel('Longueur du code')
plt.title('Longueur du codage par symbole')

plt.show()

Comme vous pouvez le constater à nouveau, plus un caractère est rare, plus il est associé à un grand nombre de bits.  Vous voyez pourquoi en faisant cela on en vient à compresser le texte?

In [ ]:
# Création d'un dictionnaire à partir de l'array
symbol_to_code = {row[0]: row[1] for row in huff_code}
phrase_conv = ""
for s in text:
    phrase_conv +=  symbol_to_code.get(s)

# Phrase reconvertie
print(f'Le codage de la phrase dans son entièreté donne: \n{phrase_conv}')

In [ ]:
# Nombre de bits moyen pour l'encodage de la phrase
print(f"Le nombre total de bits utilisé pour encoder ce texte est: {len(phrase_conv):.3f} bits")

# Nombre de bits moyen pour l'encodage de la phrase
print(f"Le nombre de bits moyen utilisé pour encoder ce texte est: {len(phrase_conv)/txt_lg:.3f} bits")

In [ ]:
H = np.sum(freq * -np.log2(freq))

print(f"Le nombre de bits moyen calculé avec l'Entropie est {H:.3f} bits")

De façon intéressant, le nombre de bits moyen utilisé pour encoder ce texte (4.058) avec le codage de Huffman est très proche de l'entropie (4.010), c'est-à-dire la valeur théorique minimale pouvant être atteinte sans perte d'information.

## Distance de Kullback-Leibler
La distance de *Kullback-Leibler (KL)* est une mesure de divergence entre deux distributions de probabilité. Elle quantifie combien une distribution de probabilité diverge d'une autre distribution de référence. En termes simples, elle mesure à quel point une distribution est différente d'une autre en utilisant l'information que chacune apporte. <br>

Formellement, pour deux distributions de probabilité $P$ et $Q$, la distance de Kullback-Leibler de $P$ par rapport à $Q$, notée $D_{KL}(P||Q)$ est définie comme suit: <br>
- $D_{KL}(P||Q)$ = $\sum_x{P(x)\, log\, \dfrac{P(x)}{Q(x)}}$ &nbsp; pour les distributions discrètes
- $D_{KL}(P||Q)$ = $\int_{-\infty}^\infty p(x)\, log\, \dfrac{p(x)}{q(x)} \mathrm{d}x$ &nbsp; pour les distributions continues
<br>
La distance KL peut être interprétée comme une mesure de l'information perdue lorsque $Q$ est utilisé pour approximer $P$. En d'autres termes, elle indique combien d'information est perdue ou combien il est "coûteux" de modéliser $P$ avec $Q$ au lieu de $P$ lui-même.

In [ ]:
#
# Exemple du calcul de la KL divergence entre 2 distributions P et Q
#

# Simulation des distributions
P = np.array([36/100, 48/100, 16/100])
Q = np.array([1/3, 0.1/3, 1/3])

# Affichage des distributions P et Q
plt.figure(figsize=(14, 5))

plt.subplot(1,2,1)
plt.bar(['X1', 'X2', 'X3'], P, alpha=0.6, label='P', color='blue')
plt.xlabel('Événements')
plt.ylabel('Probabilités')
plt.title('Distribution P')
plt.legend()

plt.subplot(1,2,2)
plt.bar(['X1', 'X2', 'X3'], Q, alpha=0.6, label='Q', color='red')
plt.xlabel('Événements')
plt.ylabel('Probabilités')
plt.title('Distribution Q')
plt.legend()

plt.show()

# Distance de Kullback-Leibler
D_KL = np.sum(P * np.log2(P / Q))

print(f'Distance de Kullback-Leibler: {D_KL:.3f} bits')


## Entropie croisée

En classification, il est courant d'utiliser des réseaux de neurones comportant une couche *softmax* en sortie. Cette couche permet de prédire une distribution de probabilité, désignée par $Q(x)$, représentant la probabilité d'appartenance à différentes classes $C$, donnée un vecteur de données $x$.

Pour entraîner un tel réseau, on utilise un "one-hot vector" comme distribution de probabilité cible, notée $P(x)$. L'objectif de l'entraînement est de déterminer les meilleurs paramètres $W$ du réseau, de manière à ce que la sortie $Q(x)$ se rapproche le plus possible de la cible $P(x)$. Autrement dit, il s'agit de minimiser la distance de Kullback-Leibler ($D_{KL}$) entre $P(x)$ et $Q(x)$. 

La distance de KL peut être reformulée comme suit :

$$
D_{KL}(P \| Q) = H(P, Q) - H(P)
$$

où $H(P)$ est l'entropie de la cible, et $H(P, Q)$ est l'entropie croisée entre la cible $P(x)$ et la prédiction $Q(x)$.

Puisque $H(P)$ est une constante qui ne dépend pas des paramètres du réseau, l'entraînement vise uniquement à minimiser l'entropie croisée $H(P, Q)$.



In [ ]:
#
# Calcul de l'entropie croisée entre 2 distributions
#

# Simulations des distributions
P = np.array([2/10, 5/10, 3/10])
Q = np.array([1/10, 6/10, 3/10])

# Représentation graphique 
x = np.arange(len(P))
width = 0.3

fig, ax = plt.subplots()
ax.bar(x - width/2, P, width, label='P (Distribution vraie)', color='blue')
ax.bar(x + width/2, Q, width, label='Q (Distribution prédite)', color='orange')

ax.set_xlabel('Événements')
ax.set_ylabel('Probabilités')
ax.set_title('Distributions de probabilité P et Q')
ax.legend()

plt.show()

# Calcul de l'entropie croisée
H_PQ = -np.sum(P * np.log(Q))
print(f"L'entropie croisée entre P et Q est : {H_PQ:.3f}")


Dans le cas d'une classification multiclasse avec des *one-hot vectors*, on a:

In [ ]:
# Simulations des distributions (vecteurs one-hot et prédictions)
P = np.array([
    [1, 0, 0],  # Classe 1
    [0, 1, 0],  # Classe 2
    [0, 0, 1],  # Classe 3
])

# Prédictions pour chaque classe (probabilités de la sortie du modèle)
Q = np.array([
    [7/10, 2/10, 1/10],  # Prédictions pour l'instance 1
    [1/10, 4/10, 5/10],  # Prédictions pour l'instance 2
    [2/10, 2/10, 6/10],  # Prédictions pour l'instance 3
])

# Représentation graphique 
x = np.arange(len(P[0]))
width = 0.3

fig, axs = plt.subplots(1, 3, figsize=(15, 5))

# Plot pour chaque instance
for i in range(len(P)):
    axs[i].bar(x - width/2, P[i], width, label='P (True One-Hot)', color='blue')
    axs[i].bar(x + width/2, Q[i], width, label='Q (Predicted)', color='orange')
    axs[i].set_xlabel(f'Instance {i+1}')
    axs[i].set_ylabel('Probabilités')
    axs[i].set_title(f'Distribution pour l\'Instance {i+1}')
    axs[i].set_xticks(x)
    axs[i].set_xticklabels(['Classe 1', 'Classe 2', 'Classe 3'])
    axs[i].legend()

plt.suptitle('Distributions de Probabilité P (One-Hot) et Q (Prédictions) pour les Instances')
plt.show()

# Calcul de l'entropie croisée pour chaque instance
H_PQ = -np.sum(P[0] * np.log(Q[0]))
print(f"L'entropie croisée totale entre P et Q pour l'instance 1 est : {H_PQ:.3f}")

H_PQ = -np.sum(P[1] * np.log(Q[1]))
print(f"L'entropie croisée totale entre P et Q pour l'instance 2 est : {H_PQ:.3f}")

H_PQ = -np.sum(P[2] * np.log(Q[2]))
print(f"L'entropie croisée totale entre P et Q pour l'instance 3 est : {H_PQ:.3f}")

print("\n On peut ainsi voir que l'instance 1 est celle dont la prédiction du modèle Q est la plus proche de la vérité terrain P") 

## Information mutuelle

On se pose à présent la question: **Quelle quantité d'information est contenue dans une variable aléatoire Y mais pas dans la variable aléatoire X ?**, ou encore **Quelle quantité d'information est partagée entre X et Y ?** La réponse réside dans l'information mutuelle entre $X$ et $Y$, qu'on notera $I(X,Y)$.

L'information correspond à la différence entre l'entropie jointe et les entropie conditionnelle, à savoir:

$$
I(X, Y) = H(X, Y) - H(Y \mid X) - H(X \mid Y).
$$

Si on développe ces termes et les combinons avec un peu d'algèbre on obtient:

$$I(X, Y) = E_{x} E_{y} \left\{ p_{X, Y}(x, y) \log\frac{p_{X, Y}(x, y)}{p_X(x) p_Y(y)} \right\}. $$

Tout cela peut être résumé au travers de ce qui suit:

* $H(X) - H(X \mid Y)$
* $H(Y) - H(Y \mid X)$
* $H(X) + H(Y) - H(X, Y)$

![Information Mutuelle](info_mutuelle.png)



Afin d'illustrer l'information mutuelle, on se servira la base de données **Automobile** de Kaggle en cherchant à déterminer l'information mutuelle entre les différentes caractéristiques des voitures et de leur prix.

In [ ]:
df = pd.read_csv("Automobile_data.csv")
df.head()

In [ ]:
X = df.copy()
y = X.pop("price")

# Encodage des variables
for colname in X.select_dtypes("object"):
    X[colname], _ = X[colname].factorize()

discrete_features = X.dtypes == int

# Remplacer les valeurs '?' par NaN dans y
y = y.replace('?', np.nan)

# Supprimer les lignes avec des valeurs manquantes dans y
X = X[y.notna()]
y = y.dropna()

In [ ]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
mi_scores[::3]  # Montre quelques variable ainsi que leur information mutuelle avec la variable prix


In [ ]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")


plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_scores)

On observe à travers le graphique au dessus, dans un ordre décroissant, l'ordre de priorité des variables quand il faut statuer sur le ***prix** de l'automobile

#### Références
[Illustration de l'Information Mutuelle](https://www.kaggle.com/code/ryanholbrook/mutual-information)
<br>
[Theorie de l'information expliquée](https://colab.research.google.com/github/d2l-ai/d2l-en-colab/blob/master/chapter_appendix-mathematics-for-deep-learning/information-theory.ipynb#scrollTo=2127a54d)